## Install Required Packages


In [42]:
!pip install pyathena
!pip install -U sagemaker -q
!pip install boto3 -q

## Import Libraries


In [43]:
import sys
import boto3
import sagemaker
from sagemaker.workflow.pipeline_context import PipelineSession
from sagemaker.session import Session


## Set Up Environment


In [4]:
import boto3
import sagemaker
from sagemaker.workflow.pipeline_context import PipelineSession

sagemaker_session = sagemaker.Session()
region = sagemaker_session.boto_region_name
role = sagemaker.get_execution_role()
pipeline_session = PipelineSession()
default_bucket = sagemaker_session.default_bucket()
model_package_group_name = "ProductRecommendationSystem"

In [44]:
# Initialize SageMaker session
sagemaker_session = sagemaker.session.Session()
region = sagemaker_session.boto_region_name
role = sagemaker.get_execution_role()
pipeline_session = PipelineSession()
model_package_group_name = f"ProductRecommendationModelPackageGroupName"

In [45]:
# Define S3 bucket and data paths
bucket = 'electronics-dataset'
prefix = 'data'
s3_train_data = f"s3://{bucket}/{prefix}/train/"
s3_val_data = f"s3://{bucket}/{prefix}/validation/"
s3_test_data = f"s3://{bucket}/{prefix}/test/"
s3_prod_data = f"s3://{bucket}/{prefix}/production/"

## Define Parameters


In [46]:
from sagemaker.workflow.parameters import (
    ParameterInteger,
    ParameterString,
    ParameterFloat,
)

processing_instance_count = ParameterInteger(name="ProcessingInstanceCount", default_value=1)
instance_type = ParameterString(name="TrainingInstanceType", default_value="ml.m5.xlarge")
model_approval_status = ParameterString(
    name="ModelApprovalStatus", default_value="PendingManualApproval"
)
train_data = ParameterString(
    name="TrainData",
    default_value=f"s3://{bucket}/{prefix}/train/train_data.csv",
)
val_data = ParameterString(
    name="ValData",
    default_value=f"s3://{bucket}/{prefix}/validation/validation_data.csv",
)
test_data = ParameterString(
    name="TestData",
    default_value=f"s3://{bucket}/{prefix}/test/test_data_with_outcome.csv",
)

acc_threshold = ParameterFloat(name="AccuracyThreshold", default_value=0.9)

## Define Processing Step for Feature Engineering


In [11]:
import os

# Create the Code directory if it doesn't exist
os.makedirs("Code", exist_ok=True)

In [47]:
%%writefile code/preprocessing.py
import argparse
import os
import pandas as pd

if __name__ == "__main__":
    base_dir = "/opt/ml/processing"

    train = pd.read_csv(f"{base_dir}/train1/train_data.csv", header=None)
    test = pd.read_csv(f"{base_dir}/test1/test_data_with_outcome.csv", header=None)
    validation = pd.read_csv(f"{base_dir}/val1/validation_data.csv", header=None)
    
    test = test[:round(len(test) * 0.8)]
    prod = test[round(len(test) * 0.8):].drop(test.columns[0], axis=1)

    train.to_csv(f"{base_dir}/train2/train.csv", header=False, index=False)
    validation.to_csv(f"{base_dir}/validation2/validation.csv", header=False, index=False)
    test.to_csv(f"{base_dir}/test2/test.csv", header=False, index=False)
    prod.to_csv(f"{base_dir}/prod2/prod.csv", header=False, index=False)

Overwriting code/preprocessing.py


In [48]:
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.workflow.steps import ProcessingStep

framework_version = "1.2-1"

sklearn_processor = SKLearnProcessor(
    framework_version=framework_version,
    instance_type="ml.m5.xlarge",
    instance_count=processing_instance_count,
    base_job_name="sklearn-process",
    role=role,
    sagemaker_session=pipeline_session,
)

processor_args = sklearn_processor.run(
    inputs=[
        ProcessingInput(source=train_data, destination="/opt/ml/processing/train1"),
        ProcessingInput(source=val_data, destination="/opt/ml/processing/val1"),
        ProcessingInput(source=test_data, destination="/opt/ml/processing/test1"),
    ],
    outputs=[
        ProcessingOutput(output_name="proc_train_out", source="/opt/ml/processing/train2"),
        ProcessingOutput(output_name="proc_validation_out", source="/opt/ml/processing/validation2"),
        ProcessingOutput(output_name="proc_test_out", source="/opt/ml/processing/test2"),
        ProcessingOutput(output_name="proc_prod_out", source="/opt/ml/processing/prod2"),
    ],
    code="code/preprocessing.py",
)

step_process = ProcessingStep(name="Process", step_args=processor_args)

INFO:sagemaker.image_uris:Defaulting to only available Python version: py3
/opt/conda/lib/python3.10/site-packages/sagemaker/workflow/pipeline_context.py:332: UserWarning: Running within a PipelineSession, there will be No Wait, No Logs, and No Job being started.
  warnings.warn(


### Training Step



In [49]:
from sagemaker.estimator import Estimator
from sagemaker.inputs import TrainingInput
from sagemaker.workflow.steps import TrainingStep

model_path = f"s3://{bucket}/{prefix}/model"
image_uri = sagemaker.image_uris.retrieve(
    framework="xgboost",
    region=region,
    version="1.0-1",
    py_version="py3",
    instance_type="ml.m5.xlarge",
)

xgb_train = Estimator(
    image_uri=image_uri,
    instance_type=instance_type,
    instance_count=1,
    output_path=model_path,
    role=role,
    sagemaker_session=pipeline_session,
)

xgb_train.set_hyperparameters(
    objective="binary:logistic",
    num_round=50,
    max_depth=5,
    eta=0.2,
    gamma=4,
    min_child_weight=6,
    subsample=0.7,
)

train_args = xgb_train.fit(
    inputs={
        "train": TrainingInput(
            s3_data=step_process.properties.ProcessingOutputConfig.Outputs["proc_train_out"].S3Output.S3Uri,
            content_type="text/csv",
        ),
        "validation": TrainingInput(
            s3_data=step_process.properties.ProcessingOutputConfig.Outputs["proc_validation_out"].S3Output.S3Uri,
            content_type="text/csv",
        ),
    }
)

step_train = TrainingStep(
    name="Train",
    step_args=train_args,
)

### Model Evaluation Step



In [51]:
%%writefile code/evaluation.py
import json
import tarfile
import pickle
import pandas as pd
import xgboost
from sklearn.metrics import accuracy_score

if __name__ == "__main__":
    model_path = "/opt/ml/processing/model/model.tar.gz"
    with tarfile.open(model_path) as tar:
        tar.extractall(path=".")
    model = pickle.load(open("xgboost-model", "rb"))

    test_path = "/opt/ml/processing/test2/test.csv"
    df = pd.read_csv(test_path, header=None)
    
    y_test = df.iloc[:, 0].to_numpy()
    df.drop(df.columns[0], axis=1, inplace=True)
    X_test = xgboost.DMatrix(df.values)

    predictions = model.predict(X_test)
    acc = accuracy_score(y_test, predictions)

    report_dict = {
        "classification_metrics": {
            "accuracy": acc,
        },
    }
    print(acc)

    output_dir = "/opt/ml/processing/evaluation"
    pathlib.Path(output_dir).mkdir(parents=True, exist_ok=True)

    evaluation_path = f"{output_dir}/evaluation.json"
    with open(evaluation_path, "w") as f:
        f.write(json.dumps(report_dict))

Overwriting code/evaluation.py


###  Evaluation Step


In [52]:
from sagemaker.processing import ScriptProcessor
from sagemaker.workflow.steps import ProcessingStep

script_eval = ScriptProcessor(
    image_uri=image_uri,
    command=["python3"],
    instance_type="ml.m5.xlarge",
    instance_count=1,
    base_job_name="script-eval",
    role=role,
    sagemaker_session=pipeline_session,
)

eval_args = script_eval.run(
    inputs=[
        ProcessingInput(
            source=step_train.properties.ModelArtifacts.S3ModelArtifacts,
            destination="/opt/ml/processing/model",
        ),
        ProcessingInput(
            source=step_process.properties.ProcessingOutputConfig.Outputs["proc_test_out"].S3Output.S3Uri,
            destination="/opt/ml/processing/test2",
        ),
    ],
    outputs=[
        ProcessingOutput(output_name="evaluation", source="/opt/ml/processing/evaluation"),
    ],
    code="code/evaluation.py",
)

evaluation_report = PropertyFile(
    name="EvaluationReport", output_name="evaluation", path="evaluation.json"
)

step_eval = ProcessingStep(
    name="Evaluate",
    step_args=eval_args,
    property_files=[evaluation_report],
)

### Model Registration Step



In [53]:
from sagemaker.model import Model
from sagemaker.workflow.model_step import ModelStep
from sagemaker.model_metrics import MetricsSource, ModelMetrics

model = Model(
    image_uri=image_uri,
    model_data=step_train.properties.ModelArtifacts.S3ModelArtifacts,
    sagemaker_session=pipeline_session,
    role=role,
)

model_metrics = ModelMetrics(
    model_statistics=MetricsSource(
        s3_uri=f"{step_eval.arguments['ProcessingOutputConfig']['Outputs'][0]['S3Output']['S3Uri']}/evaluation.json",
        content_type="application/json",
    )
)

register_args = model.register(
    content_types=["text/csv"],
    response_types=["text/csv"],
    inference_instances=["ml.t2.medium", "ml.m5.xlarge"],
    transform_instances=["ml.m5.xlarge"],
    model_package_group_name=model_package_group_name,
    approval_status=model_approval_status,
    model_metrics=model_metrics,
)

step_register = ModelStep(name="RegisterModel", step_args=register_args)

###  Conditional Step for Accuracy

In [54]:
from sagemaker.workflow.conditions import ConditionGreaterThanOrEqualTo
from sagemaker.workflow.condition_step import ConditionStep
from sagemaker.workflow.functions import JsonGet
from sagemaker.workflow.fail_step import FailStep

step_fail = FailStep(
    name="AccuracyFail",
    error_message=Join(on=" ", values=["Execution failed due to Accuracy <", acc_threshold]),
)

cond_lte = ConditionGreaterThanOrEqualTo(
    left=JsonGet(
        step_name=step_eval.name,
        property_file=evaluation_report,
        json_path="classification_metrics.accuracy",
    ),
    right=acc_threshold,
)

step_cond = ConditionStep(
    name="AccuracyCondition",
    conditions=[cond_lte],
    if_steps=[step_register],
    else_steps=[step_fail],
)

### Define Pipeline


In [55]:
from sagemaker.workflow.pipeline import Pipeline

pipeline_name = f"ProductRecommendationPipeline"
pipeline = Pipeline(
    name=pipeline_name,
    parameters=[
        processing_instance_count,
        instance_type,
        model_approval_status,
        train_data,
        val_data,
        test_data,
        acc_threshold,
    ],
    steps=[step_process, step_train, step_eval, step_cond],
)

pipeline.upsert(role_arn=role)

{'PipelineArn': 'arn:aws:sagemaker:us-east-1:676076160400:pipeline/ProductRecommendationPipeline',
 'ResponseMetadata': {'RequestId': 'e5e09c68-50a8-44bf-8d20-e278be11b7e0',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'e5e09c68-50a8-44bf-8d20-e278be11b7e0',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '97',
   'date': 'Mon, 01 Jul 2024 17:35:28 GMT'},
  'RetryAttempts': 0}}

### Start Pipeline Execution



In [57]:
execution = pipeline.start()
execution.describe()

# Wait for the pipeline execution to complete
execution.wait()

# List the steps of the pipeline execution and their statuses
steps = execution.list_steps()
for step in steps:
    print(f"Step: {step['StepName']}, Status: {step['StepStatus']}, Start Time: {step['StartTime']}, End Time: {step['EndTime']}")

WaiterError: Waiter PipelineExecutionComplete failed: Waiter encountered a terminal failure state: For expression "PipelineExecutionStatus" we matched expected path: "Failed"

### Start the Pipeline



In [58]:
# Examine the resulting model evaluation after the pipeline completes
from pprint import pprint

evaluation_json = sagemaker.s3.S3Downloader.read_file(
    f"{step_eval.arguments['ProcessingOutputConfig']['Outputs'][0]['S3Output']['S3Uri']}/evaluation.json"
)
pprint(json.loads(evaluation_json))

NoSuchKey: An error occurred (NoSuchKey) when calling the GetObject operation: The specified key does not exist.

### Define a Register Model Step to Create a Model Package


In [18]:
model_metrics = ModelMetrics(
    model_statistics=MetricsSource(
        s3_uri="{}/evaluation.json".format(step_eval.arguments["ProcessingOutputConfig"]["Outputs"][0]["S3Output"]["S3Uri"]),
        content_type="application/json"
    )
)

register_args = pytorch_model.register(
    content_types=["image/jpeg"],
    response_types=["application/json"],
    inference_instances=["ml.t2.medium", "ml.m5.xlarge"],
    transform_instances=["ml.m5.xlarge"],
    model_package_group_name=model_package_group_name,
    approval_status=model_approval_status,
    model_metrics=model_metrics,
    domain="COMPUTER_VISION"
)

step_register = ModelStep(name="ElectronicsRegisterModel", step_args=register_args)

INFO:sagemaker.processing:Uploaded code to s3://sagemaker-us-east-1-676076160400/electronics-eval-2024-07-01-15-09-11-221/source/sourcedir.tar.gz
INFO:sagemaker.processing:runproc.sh uploaded to s3://sagemaker-us-east-1-676076160400/electronics-eval-2024-07-01-15-09-11-221/source/runproc.sh


### Fail Step to Terminate the Pipeline Execution and Mark it as Failed


In [19]:
step_fail = FailStep(
    name="ElectronicsMAPFail",
    error_message=Join(on=" ", values=["Execution failed due to mAP <", map_threshold]),
)

In [20]:
cond_gte = ConditionGreaterThanOrEqualTo(
    left=JsonGet(
        step_name=step_eval.name,
        property_file=evaluation_report,
        json_path="detection_metrics.mAP50.value",
    ),
    right=map_threshold,
)

step_cond = ConditionStep(
    name="ElectronicsMAPCond",
    conditions=[cond_gte],
    if_steps=[step_register, step_create_model, step_transform],
    else_steps=[step_fail],
)

### Define a Pipeline of Parameters, Steps, and Conditions


In [40]:
pipeline_name = "ElectronicsPipeline"

pipeline = Pipeline(
    name=pipeline_name,
    parameters=[
        processing_instance_count,
        instance_type,
        model_approval_status,
        input_data,
        batch_data,
        map_threshold,
    ],
    steps=[step_process, step_train, step_eval, step_cond],
)

# Examine the pipeline definition
import json
definition = json.loads(pipeline.definition())
print(json.dumps(definition, indent=2))

# Submit the pipeline to SageMaker, start execution
pipeline.upsert(role_arn=role)
execution = pipeline.start()

# Wait for the pipeline execution to complete
execution.wait()

# List steps in the pipeline execution
execution.list_steps()

INFO:sagemaker.processing:Uploaded code to s3://sagemaker-us-east-1-676076160400/ElectronicsPipeline/code/1e4b7d6dc9f60d312a49dca13f45c47f/sourcedir.tar.gz
INFO:sagemaker.processing:runproc.sh uploaded to s3://sagemaker-us-east-1-676076160400/ElectronicsPipeline/code/54f0ef6bee583ff9186b762aaf572190/runproc.sh
INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker.processing:Uploaded code to s3://sagemaker-us-east-1-676076160400/ElectronicsPipeline/code/049bf5e9d232ff7d9a27fce88e210f56/sourcedir.tar.gz
INFO:sagemaker.processing:runproc.sh uploaded to s3://sagemaker-us-east-1-676076160400/ElectronicsPipeline/code/2c207c809cb0e0e9a1d77e5247f961f9/runproc.sh
INFO:sagemaker.processing:Uploaded code to s3://sagemaker-us-east-1-676076160400/ElectronicsPipeline/code/1e4b7d6dc9f60d312a49dca13f45c47f/sourcedir.tar.gz
INFO:sagemaker.processing:runproc.sh uploaded to s3://sagemaker-us-east-1-676076160400/ElectronicsPipeline

{
  "Version": "2020-12-01",
  "Metadata": {},
  "Parameters": [
    {
      "Name": "ProcessingInstanceCount",
      "Type": "Integer",
      "DefaultValue": 1
    },
    {
      "Name": "TrainingInstanceType",
      "Type": "String",
      "DefaultValue": "ml.m5.xlarge"
    },
    {
      "Name": "ModelApprovalStatus",
      "Type": "String",
      "DefaultValue": "PendingManualApproval"
    },
    {
      "Name": "mAPThreshold",
      "Type": "Float",
      "DefaultValue": 0.001
    }
  ],
  "PipelineExperimentConfig": {
    "ExperimentName": {
      "Get": "Execution.PipelineName"
    },
    "TrialName": {
      "Get": "Execution.PipelineExecutionId"
    }
  },
  "Steps": [
    {
      "Name": "ElectronicsProcess",
      "Type": "Processing",
      "Arguments": {
        "ProcessingResources": {
          "ClusterConfig": {
            "InstanceType": "ml.m5.xlarge",
            "InstanceCount": {
              "Get": "Parameters.ProcessingInstanceCount"
            },
            

INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker.processing:Uploaded code to s3://sagemaker-us-east-1-676076160400/ElectronicsPipeline/code/049bf5e9d232ff7d9a27fce88e210f56/sourcedir.tar.gz
INFO:sagemaker.processing:runproc.sh uploaded to s3://sagemaker-us-east-1-676076160400/ElectronicsPipeline/code/2c207c809cb0e0e9a1d77e5247f961f9/runproc.sh


ClientError: An error occurred (ValidationException) when calling the CreatePipeline operation: Unknown property reference [Parameters.InputData].

In [23]:
execution_steps = execution.list_steps()
for step in execution_steps:
    print(f"Step Name: {step['StepName']}")
    print(f"Status: {step['StepStatus']}")
    if 'FailureReason' in step['Metadata']:
        print(f"Failure Reason: {step['Metadata']['FailureReason']}")
    print("------")

Step Name: ElectronicsTrain
Status: Failed
------
Step Name: ElectronicsProcess
Status: Failed
------


In [31]:
import boto3

log_client = boto3.client('logs')

def list_log_streams(log_group_name, job_name_prefix):
    response = log_client.describe_log_streams(
        logGroupName=log_group_name,
        logStreamNamePrefix=job_name_prefix
    )
    log_streams = response.get('logStreams', [])
    return log_streams

def get_logs(log_group_name, log_stream_name):
    response = log_client.get_log_events(
        logGroupName=log_group_name,
        logStreamName=log_stream_name,
        startFromHead=True
    )
    for event in response['events']:
        print(event['message'])

def retrieve_logs_for_job(log_group_name, job_name):
    print(f"Retrieving logs for job: {job_name}")
    log_streams = list_log_streams(log_group_name, job_name)
    if not log_streams:
        print(f"No log streams found for job: {job_name}")
    for log_stream in log_streams:
        print(f"Fetching logs for log stream: {log_stream['logStreamName']}")
        get_logs(log_group_name, log_stream['logStreamName'])

# Define the log group names for processing and training jobs
processing_log_group = '/aws/sagemaker/ProcessingJobs'
training_log_group = '/aws/sagemaker/TrainingJobs'

# Define your job names
processing_job_name = 'sagemaker-xgboost-2024-06--ProfilerReport-1719730592-f9d4972a'
training_job_name = 'xgboost-2024-06-30-22-29-23-246'

# Retrieve logs for the processing job
retrieve_logs_for_job(processing_log_group, processing_job_name)

# Retrieve logs for the training job
retrieve_logs_for_job(training_log_group, training_job_name)

Retrieving logs for job: sagemaker-xgboost-2024-06--ProfilerReport-1719730592-f9d4972a
Fetching logs for log stream: sagemaker-xgboost-2024-06--ProfilerReport-1719730592-f9d4972a/algo-1-1719730632
[2024-06-30 06:57:40.695 ip-10-0-152-66.ec2.internal:7 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: /opt/ml/processing/input/profiler/signals/ProfilerReport-1719730592
[2024-06-30 06:57:41.325 ip-10-0-152-66.ec2.internal:7 INFO profiler_trial.py:67] Waiting for profiler data.
[2024-06-30 06:57:51.335 ip-10-0-152-66.ec2.internal:7 INFO profiler_trial.py:67] Waiting for profiler data.
[2024-06-30 06:58:01.343 ip-10-0-152-66.ec2.internal:7 INFO profiler_trial.py:67] Waiting for profiler data.
[2024-06-30 06:58:51.386 ip-10-0-152-66.ec2.internal:7 INFO profiler_trial.py:37] Output files of ProfilerTrial will be saved to /opt/ml/processing/output/rule
No environment variable found with name "base_trial". Will use default param value if present
No environment variable found with name "scan_inte

### Create and Schedule Data Quality Monitor


In [35]:
import boto3

s3 = boto3.client('s3')
response = s3.list_buckets()

# Print out bucket names
for bucket in response['Buckets']:
    print(bucket['Name'])

bucket_name = 'electronics-dataset'
prefix = 'your-prefix-path/'

response = s3.list_objects_v2(Bucket=bucket_name, Prefix=prefix)

if 'Contents' in response:
    for obj in response['Contents']:
        print(obj['Key'])
else:
    print(f"No objects found in {bucket_name}/{prefix}")

electronics-dataset
sagemaker-studio-676076160400-3rt69i74mgv
sagemaker-us-east-1-676076160400
No objects found in electronics-dataset/your-prefix-path/


In [36]:
from sagemaker import get_execution_role

role = get_execution_role()
print(role)

arn:aws:iam::676076160400:role/LabRole


In [37]:
import boto3

s3 = boto3.client('s3')
bucket_name = 'electronics-dataset'

# Create folders
folders = ['sagemaker/benchmark-model/batch-data/', 'sagemaker/benchmark-model/reports/']

for folder in folders:
    s3.put_object(Bucket=bucket_name, Key=(folder + '/'))

print("Folders created successfully.")

Folders created successfully.


In [38]:
import boto3
from sagemaker import get_execution_role
from sagemaker.model_monitor import DefaultModelMonitor, DatasetFormat, BatchTransformInput, MonitoringDatasetFormat, CronExpressionGenerator

role = 'arn:aws:iam::676076160400:role/LabRole'
input_data = 's3://electronics-dataset/DatafinitiElectronicsProductData.csv'
transformer_output = 's3://electronics-dataset/sagemaker/benchmark-model/output/'
batch_data = 's3://electronics-dataset/sagemaker/benchmark-model/batch-data/'
s3_report_path = 's3://electronics-dataset/sagemaker/benchmark-model/reports/'

baseline_results_uri = f"{transformer_output}/baseline_results"

my_default_monitor = DefaultModelMonitor(
    role=role,
    instance_count=1,
    instance_type='ml.m5.xlarge',
)

my_default_monitor.suggest_baseline(
    baseline_dataset=input_data,
    dataset_format=DatasetFormat.csv(header=True),
    output_s3_uri=baseline_results_uri,
    wait=True
)

data_quality_model_monitor = DefaultModelMonitor(
    role=role,
    instance_count=1,
    instance_type='ml.m5.xlarge',
)

schedule = data_quality_model_monitor.create_monitoring_schedule(
    monitor_schedule_name='ElectronicsDataQuality',
    batch_transform_input=BatchTransformInput(
        data_captured_destination_s3_uri=batch_data,
        destination="/opt/ml/processing/input",
        dataset_format=MonitoringDatasetFormat.csv(header=False),
    ),
    output_s3_uri=s3_report_path,
    statistics=f"{baseline_results_uri}/statistics.json",
    constraints=f"{baseline_results_uri}/constraints.json",
    schedule_cron_expression=CronExpressionGenerator.hourly(),
    enable_cloudwatch_metrics=True,
)

INFO:sagemaker.image_uris:Defaulting to the only supported framework/algorithm version: .
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.
INFO:sagemaker:Creating processing-job with name baseline-suggestion-job-2024-07-01-15-39-42-219


............2024-07-01 15:41:39.954762: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-07-01 15:41:39.954793: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2024-07-01 15:41:41.591716: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2024-07-01 15:41:41.591746: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2024-07-01 15:41:41.591764: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ip-10-2-105-216.ec2.internal): /proc/driver/nvidia/version does not exist
2024-07-01 15:41:41.592036: I tensor

UnexpectedStatusException: Error for Processing job baseline-suggestion-job-2024-07-01-15-39-42-219: Failed. Reason: AlgorithmError: Error: Errors occurred when analyzing your data. Please check CloudWatch logs for more details., exit code: 255

In [ ]:
import boto3
from sagemaker import get_execution_role
from sagemaker.model_monitor import DefaultModelMonitor, DatasetFormat, BatchTransformInput, MonitoringDatasetFormat, CronExpressionGenerator

role = 'arn:aws:iam::676076160400:role/LabRole'
input_data = 's3://electronics-dataset/DatafinitiElectronicsProductData.csv'
transformer_output = 's3://electronics-dataset/sagemaker/benchmark-model/output/'
batch_data = 's3://electronics-dataset/sagemaker/benchmark-model/batch-data/'
s3_report_path = 's3://electronics-dataset/sagemaker/benchmark-model/reports/'

baseline_results_uri = f"{transformer_output}/baseline_results"

# Create DefaultModelMonitor
my_default_monitor = DefaultModelMonitor(
    role=role,
    instance_count=1,
    instance_type='ml.m5.xlarge',
)

# Suggest baseline
my_default_monitor.suggest_baseline(
    baseline_dataset=input_data,
    dataset_format=DatasetFormat.csv(header=True),
    output_s3_uri=baseline_results_uri,
    wait=True
)

# Create Data Quality Model Monitor
data_quality_model_monitor = DefaultModelMonitor(
    role=role,
    instance_count=1,
    instance_type='ml.m5.xlarge',
)

# Schedule Data Quality Monitoring
schedule = data_quality_model_monitor.create_monitoring_schedule(
    monitor_schedule_name='ElectronicsDataQuality',
    batch_transform_input=BatchTransformInput(
        data_captured_destination_s3_uri=batch_data,
        destination="/opt/ml/processing/input",
        dataset_format=MonitoringDatasetFormat.csv(header=False),
    ),
    output_s3_uri=s3_report_path,
    statistics=f"{baseline_results_uri}/statistics.json",
    constraints=f"{baseline_results_uri}/constraints.json",
    schedule_cron_expression=CronExpressionGenerator.hourly(),
    enable_cloudwatch_metrics=True,
)